In [ ]:
def IntialiseEnv():
    nb_dir = os.path.split(os.getcwd())[0]
    if nb_dir not in sys.path:
        sys.path.append(nb_dir)
    invalid_path='/opt/ros/kinetic/lib/python2.7/dist-packages'
    if invalid_path in sys.path:
        sys.path.remove(invalid_path)

In [ ]:
# Libraries
import os
import sys
IntialiseEnv()
import tensorflow as tf
import numpy as np
import math
import import_ipynb
from sklearn.model_selection import train_test_split
from Data_Preparation.Data_Preparation import LoadTrainDataSetWithAugmentationFeatureExtractionModified
from Data_Preparation.Data_Preparation import LoadTestDataSetFeatureExtractionModified

In [ ]:
# Use GPU/CPU Configurations

import tensorflow as tf
print(tf.__version__)
print(tf.test.gpu_device_name())
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

In [ ]:
DataSetImages, DataSetLabels = LoadTrainDataSetWithAugmentationFeatureExtractionModified()

In [ ]:
image_train, image_valid,label_train, label_valid = train_test_split(DataSetImages,DataSetLabels,stratify=DataSetLabels,test_size=0.2,random_state=43)

In [ ]:
print(image_train.shape)

In [ ]:
test_image_array, test_image_labels = LoadTestDataSetFeatureExtractionModified()

In [ ]:
# Hyper-parameters tuning
kernel_1 = (5,5)
kernel_2 = (3,3)
pooling = (2,2)
filter_size = 32
hidden_layer = 64
dropout = 0.1
num_classes = 43

In [ ]:
def model_init_fn():
    model = None
    tf.initializers.VarianceScaling(scale=2.0)
    
    layers = [
        tf.keras.layers.Conv2D(filter_size,kernel_1, strides = 1, padding='valid',activation = tf.nn.relu),
        tf.keras.layers.MaxPool2D(pooling),
        tf.keras.layers.Dropout(dropout),
        tf.keras.layers.Conv2D(64,kernel_2, strides = 1, padding='valid', activation = tf.nn.relu),
        tf.keras.layers.MaxPool2D(pooling),
        tf.keras.layers.Dropout(dropout),
        tf.keras.layers.Conv2D(64,kernel_2, strides = 1, padding='valid', activation = tf.nn.relu),
        tf.keras.layers.MaxPool2D(pooling),
        tf.keras.layers.Dropout(dropout),
        tf.keras.layers.Conv2D(filter_size,kernel_2, strides = 1, padding='valid', activation = tf.nn.relu),
        tf.keras.layers.MaxPool2D(pooling),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(hidden_layer, activation = tf.nn.relu),
        tf.keras.layers.Dropout(dropout),
        tf.keras.layers.Dense(num_classes, activation = 'softmax')
    ]
    model = tf.keras.Sequential(layers)
    return model


model = model_init_fn()

model.compile(optimizer='sgd',
              loss='sparse_categorical_crossentropy',
              metrics=[tf.keras.metrics.sparse_categorical_accuracy])


model.fit(image_train, np.array(label_train) , batch_size=64, epochs=40, validation_data=(image_valid, np.array(label_valid)))


In [ ]:
def saveCNNModel():
    model.save('cnn_model_modified.h5')

In [ ]:
#  model._set_inputs(image_train,np.array(label_train))
saveCNNModel()

In [ ]:
def load_model():
    Model = tf.keras.models.load_model('cnn_model_modified.h5')
    return Model

In [ ]:
# del model
model._set_inputs(image_train,np.array(label_train))
model= load_model()

In [ ]:
model.evaluate(test_image_array, np.array(test_image_labels), batch_size=128)